In [2]:
import numpy as np
import pandas as pd
import pandasql as ps
import datetime
from tqdm import notebook
from datetime import datetime,date

In [3]:
biodata = pd.read_csv('biodata.csv',dtype='object')
combined = pd.read_csv('combined.csv',dtype='object')

## Step 1: Data Cleaning
To prepare the data for merging, I first clean the two datasets respectively. For question data, I drop the Committee since later we will be merging in Committee from biodata, and convert the question date to python datetime format for comparison during the merging stage.

In [4]:
def format_date(date):
    if not pd.isna(date):
        year = date.year
        if year >= 2022:
            year -= 100
        return datetime(year,date.month,date.day)

In [5]:
# clean combined data
combined.drop(columns=['Committee'],inplace=True)
combined.rename(columns={'MEP(s)':'MEP'},inplace=True)
combined['Question Date'] = pd.to_datetime(combined['Question Date'],infer_datetime_format=True).dt.date
combined.columns = combined.columns.str.replace(' ','_')

# clean biodata
biodata.rename(columns={'Party_y 1':'Party 1','Party_y 2':'Party 2','Party_y 3':'Party 3','Party_y 4':'Party 4','Party_y 5':'Party 5'},inplace=True)
date_columns = [7,8,9,10]
for i in range(len(date_columns)):
    col_name = biodata.columns[date_columns[i]]
    biodata[col_name] = pd.to_datetime(biodata[col_name],infer_datetime_format=True).dt.date
    biodata[col_name] = biodata[col_name].apply(format_date)
    biodata[col_name] = pd.to_datetime(biodata[col_name],infer_datetime_format=True).dt.date
biodata.columns = biodata.columns.str.replace(' ','_')
biodata

,MEP_ID,Term,Year,Name_original,Link,Name_scraped,Country,Birth_Date,Death_Date,Entered_Office,...,Leadership_Start_Date_6_dummy,Leadership_Start_Date_7_dummy,Leadership_End_Date_1_dummy,Leadership_End_Date_2_dummy,Leadership_End_Date_3_dummy,Leadership_End_Date_4_dummy,Leadership_End_Date_5_dummy,Leadership_End_Date_6_dummy,Leadership_End_Date_7_dummy,nameID
0,1,3,1991,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,...,1,1,0,0,0,0,0,1,1,"('georg', 'jarzembowski')"
1,1,3,1992,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,...,1,1,0,0,0,0,0,1,1,"('georg', 'jarzembowski')"
2,1,3,1993,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,...,1,1,0,0,0,0,0,1,1,"('georg', 'jarzembowski')"
3,1,3,1994,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,...,1,1,0,0,0,0,0,1,1,"('georg', 'jarzembowski')"
4,1,4,1994,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1994-07-19,...,1,1,1,0,0,0,0,1,1,"('georg', 'jarzembowski')"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31740,197566,9,2019,Witold Jan Waszczykowski,https://www.europarl.europa.eu/meps/en/197566/...,Witold Jan WASZCZYKOWSKI,Poland,1957-05-05,NaT,2019-02-07,...,0,1,0,0,0,0,0,0,1,"('witold', 'waszczykowski')"
31741,197507,9,2019,Włodzimierz Cimoszewicz,https://www.europarl.europa.eu/meps/en/197507/...,Włodzimierz CIMOSZEWICZ,Poland,1950-09-13,NaT,2019-02-07,...,0,1,0,0,0,0,0,0,1,"('wodzimierz', 'cimoszewicz')"
31742,197746,9,2019,Álvaro Amaro,https://www.europarl.europa.eu/meps/en/197746/...,Álvaro AMARO,Portugal,1953-05-25,NaT,2019-02-07,...,0,1,0,0,0,0,0,0,1,"('alvaro', 'amaro')"
31743,197468,9,2019,Özlem Demirel,https://www.europarl.europa.eu/meps/en/197468/...,Özlem DEMIREL,Germany,1984-10-03,NaT,2019-02-07,...,0,1,0,0,0,0,0,0,1,"('ozlem', 'demirel')"


## Step 2: Add Overall Tenure Start / End Date to biodata
Before merging, I also add the overall tenure start/end date. In some rare cases, a MEP might have inconsecutive tenure term. But after inspecting the data, I believe the majority of the discontinuity is due to the ordering of data. 

In [6]:
duration = biodata[['nameID','Entered_Office','Exited_Office','Tenure']]
duration = duration.fillna(date.today())
duration['Entered_Office']=pd.to_datetime(duration['Entered_Office'],infer_datetime_format=True).dt.date
duration['Exited_Office']=pd.to_datetime(duration['Exited_Office'],infer_datetime_format=True).dt.date
grouped = duration.groupby('nameID').agg(list).reset_index()
grouped['Overall_Tenure_Started'] = grouped['Entered_Office'].apply(lambda x:x[0])
grouped['Overall_Tenure_Exited'] = grouped['Exited_Office'].apply(lambda x:x[-1])
grouped = grouped[['nameID','Overall_Tenure_Started','Overall_Tenure_Exited']]
biodata_basic = biodata.iloc[:,np.r_[:13,186]]
final = biodata_basic.merge(grouped,on='nameID',how='left')
final

,MEP_ID,Term,Year,Name_original,Link,Name_scraped,Country,Birth_Date,Death_Date,Entered_Office,Exited_Office,Birth_City,Tenure,nameID,Overall_Tenure_Started,Overall_Tenure_Exited
0,1,3,1991,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,1994-07-18,Braunschweig,0,"('georg', 'jarzembowski')",1991-05-09,2009-07-13
1,1,3,1992,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,1994-07-18,Braunschweig,1,"('georg', 'jarzembowski')",1991-05-09,2009-07-13
2,1,3,1993,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,1994-07-18,Braunschweig,2,"('georg', 'jarzembowski')",1991-05-09,2009-07-13
3,1,3,1994,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1991-05-09,1994-07-18,Braunschweig,3,"('georg', 'jarzembowski')",1991-05-09,2009-07-13
4,1,4,1994,Georg Jarzembowski,https://www.europarl.europa.eu/meps/en/1/GEORG...,Georg JARZEMBOWSKI,Netherlands,1947-03-02,NaT,1994-07-19,1999-07-19,Braunschweig,3,"('georg', 'jarzembowski')",1991-05-09,2009-07-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31740,197566,9,2019,Witold Jan Waszczykowski,https://www.europarl.europa.eu/meps/en/197566/...,Witold Jan WASZCZYKOWSKI,Poland,1957-05-05,NaT,2019-02-07,NaT,Piotrkow Trybunalski,0,"('witold', 'waszczykowski')",2019-02-07,2022-02-11
31741,197507,9,2019,Włodzimierz Cimoszewicz,https://www.europarl.europa.eu/meps/en/197507/...,Włodzimierz CIMOSZEWICZ,Poland,1950-09-13,NaT,2019-02-07,NaT,Warszawa,0,"('wodzimierz', 'cimoszewicz')",2019-02-07,2022-02-11
31742,197746,9,2019,Álvaro Amaro,https://www.europarl.europa.eu/meps/en/197746/...,Álvaro AMARO,Portugal,1953-05-25,NaT,2019-02-07,NaT,Coimbra,0,"('alvaro', 'amaro')",2019-02-07,2022-02-11
31743,197468,9,2019,Özlem Demirel,https://www.europarl.europa.eu/meps/en/197468/...,Özlem DEMIREL,Germany,1984-10-03,NaT,2019-02-07,NaT,Malatya,0,"('ozlem', 'demirel')",2019-02-07,2022-02-11


## Step 3: Merge in Basic Info (Country, Birth Day, Birth City...)
I first merge in all the basic info (such as country, birthday, that won't change over time. Instead of pandas merge function, I choose SQL here, which adds more flexibility and also boost the runtime. 

In [7]:
combined['Question_Date'] = pd.to_datetime(combined['Question_Date'],infer_datetime_format=True).dt.date
final.rename(columns={'nameID':'name_ID','Year':'year'},inplace=True)
sqlcode = '''
SELECT *
FROM combined
LEFT JOIN final 
ON combined.nameID = final.name_ID AND combined.Year = final.year
WHERE date(combined.Question_Date) between date(final.Entered_Office) and date(final.Exited_Office)
'''
final = ps.sqldf(sqlcode,locals())
final.drop(columns=['name_ID','year'],inplace=True)
final

,Unnamed:_0,Subject,Question_Type,Rule,MEP,Parties,Question_Date,Question_ID,Question_Link,Question_Text,...,Name_scraped,Country,Birth_Date,Death_Date,Entered_Office,Exited_Office,Birth_City,Tenure,Overall_Tenure_Started,Overall_Tenure_Exited
0,0,Community and international environmental ed...,Written,None,Carmen Díez de Rivera Icaza,PSE,1994-09-01,E-1700/94,https://www.europarl.europa.eu/sides/getDoc.do...,Chapter 36 of Agenda 21 (Unced) echoes the Tb...,...,Carmen DÍEZ DE RIVERA ICAZA,Spain,1942-08-29,1999-11-30,1994-07-19,1999-01-31,Madrid,7,1987-06-07,1999-01-31
1,1,Treatment of solid urban waste on Majorca,Written,None,Antoni Gutiérrez Díaz,GUE,1994-09-01,E-1704/94,https://www.europarl.europa.eu/sides/getDoc.do...,The Autonomous Community of the Balearic Isla...,...,Antoni GUTIÉRREZ DÍAZ,Spain,1929-01-19,2006-06-10,1994-07-19,1999-07-19,Premiá de Mar (Barcelona),7,1987-06-07,1999-07-19
2,2,Administrative costs of development aid from...,Written,None,Winifred Ewing,ARE,1994-09-01,E-1761/94,https://www.europarl.europa.eu/sides/getDoc.do...,Has the Commission ever calculated the percen...,...,Winifred M. EWING,United Kingdom,1929-10-07,None,1994-07-19,1999-07-19,Glasgow,15,1979-07-17,1999-07-19
3,3,Imports from Poland of diseased ewes,Written,None,Winifred Ewing,ARE,1994-09-01,E-1762/94,https://www.europarl.europa.eu/sides/getDoc.do...,Is the Commission aware of lorry loads of old...,...,Winifred M. EWING,United Kingdom,1929-10-07,None,1994-07-19,1999-07-19,Glasgow,15,1979-07-17,1999-07-19
4,5,Aids for the processing of apricots,Written,None,Francisca Bennasar Tous,PPE,1994-09-12,E-1943/94,https://www.europarl.europa.eu/sides/getDoc.do...,The apricot is a quintessentially Mediterrane...,...,Francisca BENNASAR TOUS,Spain,1943-11-12,None,1994-07-19,1999-07-19,Baleares,0,1994-07-19,1999-07-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164073,31,Electricity transmission lines,Written,None,David Hammerstein Mintz,Verts/ALE,2006-01-23,E-0297/06,https://www.europarl.europa.eu/sides/getDoc.do...,A new electricity transmission line (400 kW) ...,...,David HAMMERSTEIN,Spain,1955-09-23,None,2004-07-20,2009-07-13,Los Angeles,2,2004-07-20,2009-07-13
164074,32,Planned time frame for conclusion of inves...,Written,130.0,Janusz Władysław Zemke,S&D,2014-10-16,P-008070/2014,https://www.europarl.europa.eu/doceo/document/...,I have been approached in my capacity as a mem...,...,Janusz ZEMKE,Poland,1949-02-24,None,2014-01-07,2019-01-07,Kowalewo,5,2009-07-14,2019-01-07
164075,33,Vietnamese democracy,Written,None,Claude Moraes,PSE,2006-08-17,E-3715/06,https://www.europarl.europa.eu/sides/getDoc.do...,Is the Commission aware of the case of Nguyen...,...,Claude MORAES,United Kingdom,None,None,2004-07-20,2009-07-13,Not listed,7,1999-07-20,2020-01-31
164076,34,German aircraft on a Frontex mission over ...,Written,117.0,Josef Weidenholzer,S&D,2012-10-31,P-009944/2012,https://www.europarl.europa.eu/sides/getDoc.do...,The Spanish organisation ‘Andalucía acoge’ pu...,...,Josef WEIDENHOLZER,Austria,1950-06-03,None,2011-12-14,2014-06-30,St. Florian am Inn,1,2011-12-14,2019-01-07


## Step 4: Merge in Public Office Positions based on NameID and Question_Date
Below is the "mega" function that merge in all positions corresponds to the MEP during the time he/she asks the question. The logic here is merge in Committee/Subcommittee/Delegation/Party seperately, and for each of them, I stack their positions and start/end time back to back rather than a one-hot in the original question data, then merge with the question data on NameID and time intervals. 

### Function

In [8]:
def process(name,i,final):
    temp1 = pd.DataFrame()
    
    # Create seperate tables for each public office positions
    for i in range(1,i):
        if name=='Leadership':
            temp2 = biodata[['nameID',name+'_Group_'+str(i),name+'_Position_'+str(i),name+'_Start_Date_'+str(i),name+'_End_Date_'+str(i)]]
            temp2 = temp2.rename(columns={name+'_Group_'+str(i):name+'_Group',
                                          name+'_Position_'+str(i):name+'_Position',
                                          name+'_Start_Date_'+str(i):'Start_Date',
                                          name+'_End_Date_'+str(i):'End_Date'})
        else:
            temp2 = biodata[['nameID',name+'_'+str(i),name+'_Start_Date_'+str(i),name+'_End_Date_'+str(i)]]
            temp2 = temp2.rename(columns={name+'_'+str(i):name,
                                          name+'_Start_Date_'+str(i):'Start_Date',
                                          name+'_End_Date_'+str(i):'End_Date'})
        temp1 = pd.concat([temp1,temp2])
        temp1.drop_duplicates(inplace=True)
        temp1.reset_index(inplace=True,drop=True)
        for j in range(temp1.shape[0]):
            start_date = str(temp1.loc[j,'Start_Date'])
            if not pd.isna(start_date) and '/' in start_date:
                split = start_date.split('/')
                day = int(split[0])
                month = int(split[1])
                if int(split[2])+2000>2022:
                    year = int(split[2])+1900
                else:
                    year = int(split[2])+2000
                temp1.loc[j,'Start_Date'] = datetime(year,month,day)
            
            end_date = str(temp1.loc[j,'End_Date'])
            if not pd.isna(end_date) and '/' in end_date:
                split = end_date.split('/')
                day = int(split[0])
                month = int(split[1])
                if int(split[2])+2000>2022:
                    year = int(split[2])+1900
                else:
                    year = int(split[2])+2000
                temp1.loc[j,'End_Date'] = datetime(year,month,day)
        temp1['Start_Date'] = pd.to_datetime(temp1['Start_Date'],infer_datetime_format=True).dt.date
        temp1['End_Date'] = pd.to_datetime(temp1['End_Date'],infer_datetime_format=True).dt.date
        
    grouped = temp1.groupby(by=['nameID']).count()
    # If an entry has a start date but not an end date, fill in current date for merging purposes
    fillToday = np.array(grouped[grouped['Start_Date'] > grouped['End_Date']].reset_index()['nameID'])
    # If a MEP has no committee at all, we keep the entry even all the following column have na values
    if name == 'Leadership':
        keepNA = np.array(grouped[grouped['Leadership_Group']==0].reset_index()['nameID'])
    else:
        keepNA = np.array(grouped[grouped[name]==0].reset_index()['nameID'])
    temp1.reset_index(drop=True,inplace=True)
    
    for i in range(temp1.shape[0]):
        if temp1.loc[i,'nameID'] in fillToday and pd.isna(temp1.loc[i,'End_Date']):
            temp1.loc[i,'End_Date'] = date.today()
        if name == 'Leadership':
            if temp1.loc[i,'nameID'] not in keepNA and pd.isna(temp1.loc[i,'Leadership_Group']):
                temp1 = temp1.drop([i])
        else:
            if temp1.loc[i,'nameID'] not in keepNA and pd.isna(temp1.loc[i,name]):
                temp1 = temp1.drop([i])
    temp1.rename(columns={'nameID':'name_ID'},inplace=True)
    
    sqlcode = '''
    SELECT *
    FROM final
    LEFT JOIN temp1
    ON final.nameID = temp1.name_ID
    WHERE date(final.Question_Date) between date(temp1.Start_Date) and date(temp1.End_Date)
    '''
    temp3 = ps.sqldf(sqlcode,locals())
    temp3.drop(columns=['name_ID'],inplace=True)
    
    if name == 'Leadership':
        temp3 = temp3[['Question_ID','Leadership_Group','Leadership_Position','Start_Date','End_Date']].groupby(['Question_ID']).agg('; '.join).reset_index()
    else:
        temp3 = temp3[['Question_ID',name,'Start_Date','End_Date']].groupby(['Question_ID']).agg('; '.join).reset_index()
    temp3.rename(columns={'Question_ID':'QuestionID'},inplace=True)
    sqlcode = '''
    SELECT *
    FROM final
    LEFT JOIN temp3
    ON final.Question_ID = temp3.QuestionID
    '''
    final = ps.sqldf(sqlcode,locals())
    final.drop(columns=['QuestionID'],inplace=True)
    
    if name == 'Leadership':
        name_dummies_1 = final['Leadership_Group'].str.split(';', expand=True)
        for i in range(name_dummies_1.shape[1]):
            name_dummies_1 = name_dummies_1.rename(columns={i:'Leadership_Group_'+str(i+1)})
        final = final.join(name_dummies_1)
        name_dummies_2 = final['Leadership_Position'].str.split(';', expand=True)
        for i in range(name_dummies_2.shape[1]):
            name_dummies_2 = name_dummies_2.rename(columns={i:'Leadership_Position_'+str(i+1)})
        final = final.join(name_dummies_2)
    else:
        name_dummies = final[name].str.split(';', expand=True)
        for i in range(name_dummies.shape[1]):
            name_dummies = name_dummies.rename(columns={i:name+'_'+str(i+1)})
        final = final.join(name_dummies)

    start_dummies = final['Start_Date'].str.split(';', expand=True)
    for i in range(start_dummies.shape[1]):
        start_dummies = start_dummies.rename(columns={i:name+'_Start_Date_'+str(i+1)})
    final = final.join(start_dummies)

    end_dummies = final['End_Date'].str.split(';', expand=True)
    for i in range(end_dummies.shape[1]):
        end_dummies = end_dummies.rename(columns={i:name+'_End_Date_'+str(i+1)})
    final = final.join(end_dummies)
    
    if name == 'Leadership':
        final.drop(columns=['Leadership_Group','Leadership_Position','Start_Date','End_Date'],inplace=True)
    else:
        final.drop(columns=[name,'Start_Date','End_Date'],inplace=True)
    return final

### Apply the Merging Function

In [8]:
column_dict = {'Committee':11,'Subcommittee':8,'Delegation':9,'National_Party':5,'Party':6,'Leadership':8}
for i in notebook.tqdm(range(len(column_dict))):
    key = list(column_dict.keys())[i]
    final = process(key,column_dict[key],final)

### Output the merged table as a csv file

In [9]:
final.to_csv("merged_table.csv")